### Tools In Langchain
 1. How To Create Tools
 2. How to use built-in tools and toolkits
 3. How to use chat models to call tools
 4. How to pass tool outputs to chat models

 - This @tool decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description - so a docstring MUST be provided.

In [2]:
#@tool decrator 

from langchain_core.tools import tool 

@tool
def division(a:int, b: int) -> int:
    """Divivde two numbers"""
    return a/b 


In [3]:
print(division.name)
print(division.description)
print(division.args)

division
Divivde two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [4]:
#async implmentation 
@tool

async def multiply(a:int, b: int) -> int:
    """Multiply two numbers"""
    return a * b 


In [5]:
from typing import Annotated, List 

@tool 
def multiply_by_max(
    a: Annotated[int, "A integer value"],
    b: Annotated[List[int], "List of integers over which to take maximum"]
) -> int:
    """ Multiply a by maximum of b """
    
    return a * max(b)


In [6]:
print(multiply_by_max.args)

{'a': {'description': 'A integer value', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'List of integers over which to take maximum', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}


#### Structured Tool
 - The StructuredTool.from_function class method provides a bit more configurability than the @tool decorator, without requiring much additional code.

In [7]:
from langchain_core.tools import StructuredTool

def multiply(a:int, b: int) -> int:
    """Multiply two numbers"""
    return a * b 


async def amultiply(a:int, b: int) -> int:
    """Multiply two numbers"""
    return a * b 

In [8]:
calculator = StructuredTool.from_function(func= multiply, coroutine= amultiply)

print(calculator.invoke({"a": 2,"b": 5}))
print(await calculator.ainvoke({"a": 5, "b": 6}))

10
30


### Inbuilt Tools

In [9]:
#Wikipedia 
from langchain_community.tools import WikipediaQueryRun 
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results = 3, doc_content_chars_max = 300)
tool = WikipediaQueryRun(api_wrapper= api_wrapper)

In [10]:
print(tool.invoke({"query": "Langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis a


In [11]:
#TavilySearch 
from dotenv import load_dotenv 
import os 
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [12]:
from langchain_tavily import TavilySearch 

tool = TavilySearch(max_results= 5, 
                    topics= 'general',
                    # include_answer=False,
                    # include_raw_content=False,
                    # include_images=False,
                    # include_image_descriptions=False,
                    # search_depth="basic",
                    # time_range="day",
                    # include_domains=None,
                    # exclude_domains=None
                )


In [13]:
tool.invoke("What is recent AI news?")

{'query': 'What is recent AI news?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.artificialintelligence-news.com/',
   'title': 'AI News | Latest AI News, Analysis & Events',
   'content': 'Gen AI makes no financial difference in 95% of cases · Google Cloud unveils AI ally for security teams · New Nvidia Blackwell chip for China may outpace H20',
   'score': 0.98504,
   'raw_content': None},
  {'url': 'https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=ASWzDAj9zRjmFQPDkc1CMXZBH7CgxYWMAiGeEMJoILFbbrY-9VmI5R7He1DD&gaa_ts=68a6e46c&gaa_sig=S1D8BnNR5zvzvGs0mlMuKvsop307yrhx9QBHMzdGpwDL42CEdAhFyomGGteuKrVZGnAVuVjpAExagztfk6RTcA%3D%3D',
   'title': 'Artificial Intelligence - Latest AI News and Analysis',
   'content': 'Artificial Intelligence · Meta Freezes AI Hiring After Blockbuster Spending Spree · GM Raided Silicon Valley to Build Its New AI Team. · Google Is Beating Apple',
   'score': 0.9834,
   'raw_content': None},
  {'url': 'https://techc

In [14]:
#Arxiv 
from langchain_community.utilities import ArxivAPIWrapper

tool = ArxivAPIWrapper()

In [15]:
tool.run("1706.03762")


'Published: 2023-08-02\nTitle: Attention Is All You Need\nAuthors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin\nSummary: The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-to-French\ntranslation task, our model establis

### Call Tool With LLM Model

In [16]:
#Wikipedia 
from langchain_community.tools import WikipediaQueryRun 
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results = 3, doc_content_chars_max = 300)
wiki_tool = WikipediaQueryRun(api_wrapper= api_wrapper)

In [17]:
from langchain_tavily import TavilySearch 

tavily_tool = TavilySearch(max_results= 5, 
                    topics= 'general',
                    # include_answer=False,
                    # include_raw_content=False,
                    # include_images=False,
                    # include_image_descriptions=False,
                    # search_depth="basic",
                    # time_range="day",
                    # include_domains=None,
                    # exclude_domains=None
                )


In [18]:
#Custom Tool 
from langchain_core.tools import tool 

@tool 
def add(a : int, b: int) -> int:
    """Add two numbers"""
    return a + b 

@tool
def multiply(a: int, b: int) -> tool:
    """Multiply two numbers"""
    return a * b

In [19]:
print(wiki_tool.name, tavily_tool.name)

wikipedia tavily_search


In [20]:
#List of tools 
tools = [wiki_tool, tavily_tool, add, multiply]

In [21]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\PythonCSV\\CSV_file\\LangChainBasics\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=300)),
 TavilySearch(max_results=5, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'), api_base_url=None)),
 StructuredTool(name='add', description='Add two numbers', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000002470F33B380>),
 StructuredTool(name='multiply', description='Multiply two numbers', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000002470DDFBE20>)]

In [22]:
#Create LLM Model 
from langchain_groq import ChatGroq 
llm = ChatGroq(model = "deepseek-r1-distill-llama-70b")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002470F5B2F90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002470F5B3B60>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
#Binding LLM with Tool 
llm_with_tool = llm.bind_tools(tools) 

In [24]:
llm_with_tool

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002470F5B2F90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002470F5B3B60>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'wikipedia', 'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'query to look up on wikipedia', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'tavily_search', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. It not only retrieves URLs and snippets,

In [34]:
from langchain_core.messages import HumanMessage
question = "What is 2 * 5?"
messages = [HumanMessage(question)]
response = llm_with_tool.invoke(question)
print(response.tool_calls)


[{'name': 'multiply', 'args': {'a': 2, 'b': 5}, 'id': 'fewhat3by', 'type': 'tool_call'}]


In [36]:

for tool_call in response.tool_calls:
    selected_tool = {"add": add, "multiply": multiply, "wikipedia": wiki_tool, "tavily_search": tavily_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

[HumanMessage(content='What is 2 * 5?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='10', name='multiply', tool_call_id='fewhat3by'),
 ToolMessage(content='10', name='multiply', tool_call_id='fewhat3by')]

In [37]:
llm_with_tool.invoke(messages)

AIMessage(content='The product of multiplying 2 and 5 is 10.\n\n', additional_kwargs={'reasoning_content': "Alright, so I need to figure out what 2 multiplied by 5 is. Hmm, I remember from basic math that multiplication is like adding a number to itself a certain number of times. So, 2 times 5 should mean adding 2 five times. Let me see: 2 + 2 is 4, plus another 2 is 6, then 8, and finally 10. So, 2 multiplied by 5 equals 10. I think that's right because when I add 2 five times, I get 10. Yeah, that makes sense. I don't see any mistakes in that calculation.\n", 'tool_calls': [{'id': '1tg2bpg14', 'function': {'arguments': '{"a":2,"b":5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 200, 'prompt_tokens': 1816, 'total_tokens': 2016, 'completion_time': 0.736724837, 'prompt_time': 0.172824869, 'queue_time': 0.046017771, 'total_time': 0.909549706}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_76307ac09b', 'se

In [38]:
from langchain_core.messages import HumanMessage

query = "What is langchain and what is 5 * 15?"
messages = [HumanMessage(query)]

ai_msg = llm_with_tool.invoke(messages)
ai_msg

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, so the user is asking two things: "What is langchain?" and "What is 5 * 15?" I need to address both parts. \n\nFirst, for the "What is langchain?" part, I should probably use a tool that can provide information. The options are the Wikipedia tool or the Tavily search tool. Since LangChain is a known library in the AI and machine learning space, I think the Tavily search would be more appropriate because it\'s optimized for current and comprehensive results. I\'ll set the search query to "langchain" and maybe include some parameters to get the most relevant information.\n\nSecond, for the multiplication part, "5 * 15," that\'s a straightforward math problem. I can use the multiply function provided. It requires two integers, a and b, so I\'ll set a to 5 and b to 15.\n\nI should make sure to present both answers clearly, one after the other. First, the information about LangChain, then the result of the multiplication. 

In [39]:
ai_msg.tool_calls

[{'name': 'tavily_search',
  'args': {'end_date': None,
   'exclude_domains': [],
   'include_domains': [],
   'include_favicon': False,
   'include_images': False,
   'query': 'langchain',
   'search_depth': 'advanced',
   'start_date': None,
   'time_range': None,
   'topic': 'general'},
  'id': 'amc9tf7ka',
  'type': 'tool_call'}]

In [40]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply,"wikipedia":wiki_tool,"tavily_search":tavily_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

[HumanMessage(content='What is langchain and what is 5 * 15?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='{"query": "langchain", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://medium.com/data-engineering-with-dremio/crash-course-on-developing-ai-applications-with-langchain-b6b25d0e6ea5", "title": "Crash Course on Developing AI Applications with LangChain", "content": "By providing a modular approach, LangChain allows you to compose **prompt templates, chains, memory, and agents** to build flexible and scalable solutions. Instead of manually handling prompts, API calls, and responses, LangChain provides a structured way to **chain together different components** such as prompts, memory, and external tools. Now that we have LangChain installed, let’s explore how to use it for interacting with LLMs, structuring prompts, and building simple AI workflows. Now that you understand the basics of LangChain — connecting to LLMs, st

In [41]:
response1=llm_with_tool.invoke(messages)

In [42]:
response1

AIMessage(content='LangChain is a framework designed to build applications using large language models (LLMs) by integrating them with external data sources and creating structured workflows. It provides tools for developers to create prompt templates, agents, and cognitive architectures, enabling the development of LLM-powered applications efficiently.\n\nAs for the multiplication, 5 multiplied by 15 equals 75.\n\nStep-by-step explanation:\n1. **Understanding LangChain**: LangChain is a framework that helps developers build applications by connecting LLMs with external components, allowing for structured workflows and integrations.\n2. **Multiplication Calculation**: \n   - 5 * 15 can be broken down into (5 * 10) + (5 * 5) = 50 + 25 = 75.\n   - Alternatively, adding 15 five times: 15 + 15 + 15 + 15 + 15 = 75.\n\nFinal Answer:\n5 * 15 = 75.', additional_kwargs={'reasoning_content': "Okay, so I need to figure out what LangChain is and calculate 5 multiplied by 15. Let's start with LangC

In [47]:
from langchain_core.messages import HumanMessage

query = "What is 5 * 15 and also What is Agentic AI?"
messages = [HumanMessage(query)]

ai_msg = llm_with_tool.invoke(messages)
ai_msg

AIMessage(content="Sure! Let's tackle both of your questions:\n\n1. **Math Calculation:**\n   - **5 * 15 = 75**\n\n2. **Information about Agentic AI:**\n   Let me search for information about Agentic AI.\n\n", additional_kwargs={'reasoning_content': 'Okay, so the user is asking two things here: first, they want to know what 5 multiplied by 15 is, and second, they\'re asking about Agentic AI. Let me break this down step by step.\n\nStarting with the math part, multiplying 5 by 15. I remember that 5 times 10 is 50, and then 5 more 5s would be 25, so adding those together gives 75. That seems straightforward.\n\nNow, for the Agentic AI part. I\'m not exactly sure what that is. I think it might be related to AI, but I\'m not certain. Maybe it\'s a company or a specific AI model. I should look it up to get accurate information. I\'ll use the Tavily search tool for this since it\'s good for finding current and comprehensive information.\n\nI need to set up the tool call properly. The functio

In [48]:
ai_msg.tool_calls

[{'name': 'tavily_search',
  'args': {'end_date': None,
   'exclude_domains': [],
   'include_domains': [],
   'include_favicon': False,
   'include_images': False,
   'query': 'Agentic AI',
   'search_depth': 'advanced',
   'start_date': None,
   'time_range': None,
   'topic': 'general'},
  'id': '8fra71ckd',
  'type': 'tool_call'}]

In [49]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply,"wikipedia":wiki_tool,"tavily_search":tavily_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

[HumanMessage(content='What is 5 * 15 and also What is Agentic AI?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='{"query": "Agentic AI", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://en.wikipedia.org/wiki/Agentic_AI", "title": "Agentic AI - Wikipedia", "content": "Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks without human intervention. The independent systems automatically respond to conditions, to produce process results. The field is closely linked to agentic automation, also known as agent-based process management systems, when applied to process automation. Applications include software development, customer support, cybersecurity and business intelligence. [...] The core concept of agentic AI is the use of _AI agents_ to perform automated tasks but without human intervention.( While robotic process automation (RPA) and AI agents can be p

In [50]:
llm_with_tool.invoke(messages)

AIMessage(content='The product of 5 and 15 is 75. Agentic AI refers to artificial intelligence systems designed to operate autonomously, making decisions and performing tasks with minimal human intervention. These systems utilize advanced techniques like natural language processing and machine learning to adaptively achieve their goals, distinguishing them from traditional AI by their ability to handle complex tasks independently.', additional_kwargs={'reasoning_content': 'Okay, so I need to figure out what the user is asking for. They provided a prompt that says "What is 5 * 15 and also What is Agentic AI?". Hmm, so they\'re asking two separate questions here. The first one is a simple math problem, and the second is about a type of AI.\n\nFirst, let\'s tackle the math question: 5 multiplied by 15. That seems straightforward. I remember that 5 times 10 is 50, and 5 times 5 is 25, so adding those together gives 75. So, 5 * 15 equals 75. That should be easy enough.\n\nNow, the second pa

In [51]:
result = llm_with_tool.invoke(messages)
print(result.content)

The result of 5 multiplied by 15 is 75.

Agentic AI refers to a class of artificial intelligence systems designed to operate autonomously with minimal human intervention. These systems utilize advanced AI techniques such as natural language processing and machine learning to make decisions, perform tasks, and adapt to various situations without constant supervision. Unlike traditional AI models that follow predefined rules, Agentic AI can learn, reason, and execute complex workflows, making it highly effective in applications like customer support, cybersecurity, and business automation. It builds upon other AI technologies, focusing on autonomous decision-making and goal-oriented actions, thus advancing AI capabilities beyond content generation to task execution.


In [63]:
from langchain_core.messages import HumanMessage

query = "What is 25 * 5 and also What is Agentic AI and summarize the recent climate change news?"
messages = [HumanMessage(query)]

ai_msg = llm_with_tool.invoke(messages)
ai_msg

AIMessage(content='I will help you with each of your questions using the appropriate tools.\n\n1. For the multiplication:\n', additional_kwargs={'reasoning_content': 'Alright, so the user asked three things: 25 multiplied by 5, what Agentic AI is, and a summary of recent climate change news. Let me break this down.\n\nFirst, the multiplication. That\'s straightforward. 25 times 5 is 125. I can use the multiply tool for that.\n\nNext, explaining Agentic AI. Since it\'s a specific term, I\'ll need to look it up. I\'ll use the Wikipedia tool with the query "Agentic AI" to get the details.\n\nLastly, the recent climate change news. For this, I should use the Tavily search tool. I\'ll set the start date to the last 30 days to ensure the information is current. I\'ll keep the search depth as advanced to get comprehensive results and make sure to exclude any irrelevant domains if needed.\n\nI need to structure the responses clearly, separating each part so the user can easily understand each 

In [64]:

for tool_call in response.tool_calls:
    selected_tool = {"add": add, "multiply": multiply, "wikipedia": wiki_tool, "tavily_search": tavily_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

[HumanMessage(content='What is 25 * 5 and also What is Agentic AI and summarize the recent climate change news?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='10', name='multiply', tool_call_id='fewhat3by')]

In [65]:
llm_with_tool.invoke(messages)

AIMessage(content='Here are the requested results:\n\n1. **Multiplication Result**:  \n   25 multiplied by 5 equals **125**.\n\n2. **Agentic AI**:  \n   Agentic AI is a type of artificial intelligence designed to perform tasks that typically require human intelligence. It is focused on enabling machines to learn, reason, and interact with their environment in ways that simulate human-like intelligence.\n\n3. **Recent Climate Change News**:  \n   According to recent reports, global efforts to reduce carbon emissions are intensifying. The latest climate change summit emphasized the importance of international cooperation to meet Paris Agreement targets. Additionally, advancements in renewable energy technologies are showing promise in reducing greenhouse gas emissions.', additional_kwargs={'reasoning_content': 'Alright, so I need to figure out how to respond to the user\'s query. They asked three things: what is 25 multiplied by 5, what Agentic AI is, and a summary of recent climate chan